# **Introduction**
* This is my first time to join kaggle competion and get sarted in the nlp field. I still have some problems in this notebook, so I would be very gtateful for anyone to help me solve the problems or suggest me improvements.
* I tried different models here. Including tf-idf + logistic regression/decision tree, word2vec + logistic regression, embedding + cnn, word2vec + cnn, bert. The result shows that the best accuracy is bert, next is logistic regression, then is word2vec+cnn.
* When I wrote this kernel, I learned much from kernels below. The ideas from these kernels inspired me a lot, and some of my codes came from them. Thanks a lot to them!
* [https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert](http://)
* [https://www.kaggle.com/xhlulu/disaster-nlp-keras-bert-using-tfhub](http://)
* [https://www.kaggle.com/shahules/basic-eda-cleaning-and-glove](http://)
* [https://blog.csdn.net/qq_43522113/article/details/89888036?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522160680506719721942257729%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fblog.%2522%257D&request_id=160680506719721942257729&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~blog~first_rank_v1~rank_blog_v1-7-89888036.pc_v1_rank_blog_v1&utm_term=%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB%E9%A1%B9%E7%9B%AE%E5%AE%9E%E8%B7%B5](http://)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
import re
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import time
from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
from keras import losses
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

# Getting Data

In [ ]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
submit_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

In [ ]:
train_data[train_data['text'].isna()]

In [ ]:
train_data.info()

In [ ]:
train_data.groupby('target').count()

# Data Exploring

In [ ]:
%matplotlib inline

In [ ]:
piedata = train_data['target']
plt.figure(figsize=(6,6))
piedata.value_counts().plot(kind = 'pie',autopct = '%.2f%%')

More tweets with class 0 ( No disaster).  Accord with reality. 

In [ ]:
num_words_0 = train_data[train_data['target']==0]['text'].apply(lambda x: len(x.split()))
num_words_1 = train_data[train_data['target']==1]['text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(12,6))
sns.kdeplot(num_words_0, shade=True, color = 'b').set_title('Kernel distribution of number of words')
sns.kdeplot(num_words_1, shade=True, color = 'r')
plt.legend(labels=['0_no disaster', '1_disaster'])

In [ ]:
len_word_0 = train_data[train_data['target']==0]['text'].str.split().map(lambda x: [len(i) for i in x])
ave_len_0 = len_word_0.map(lambda x: np.mean(x))
len_word_1 = train_data[train_data['target']==1]['text'].str.split().map(lambda x: [len(i) for i in x])
ave_len_1 = len_word_1.map(lambda x: np.mean(x))
plt.figure(figsize=(12,6))
sns.kdeplot(ave_len_0, shade=True, color='b').set_title('Kernel distribution of average words lenth')
sns.kdeplot(ave_len_1, shade=True, color='r')
plt.legend(labels=['0_no disaster', '1_disaster'])

Both the distribution of words number and words average lenth of two  class are similiar. Class 1(disaster) trends to have more words and longer words length. Accord with reality.

# **Cleaning Data**

In [ ]:
data = pd.concat([train_data, submit_data])
data.shape

In [ ]:
data['text'] = data['text'].apply(lambda x: re.sub(re.compile(r'https?\S+'), '', x))
data['text'] = data['text'].apply(lambda x: re.sub(re.compile(r'[\//:,.!?@&\-\'\`\"\_\n\#]'), ' ', x))
data['text'] = data['text'].apply(lambda x: re.sub(re.compile(r'<.*?>'), '', x))
data['text'] = data['text'].apply(lambda x: re.sub(re.compile("["
                           u"\U0001F600-\U0001F64F"  
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE), '', x))
data['text'] = data['text'].apply(lambda x: re.sub(re.compile(r'\d'), '', x))
data['text'] = data['text'].apply(lambda x: re.sub(re.compile(r'[^\w]'), ' ', x))
data['text'] = data['text'].str.lower()

In [ ]:
'''
text_series = data.loc[:,'text']
for i in range(len(text_series)):
    content = text_series.iloc[i]
    textblob = TextBlob(content)
    text_series.iloc[i] = textblob.correct()
'''

# **Feature Engineering 1st. TF-IDF**

In [ ]:
clean_train = data[0:train_data.shape[0]]
clean_submit = data[train_data.shape[0]:-1]

X_train, X_test, y_train, y_test = train_test_split(clean_train['text'], clean_train['target'],
                                                   test_size = 0.2, random_state = 4)

In [ ]:
def tfidf(words):
    tfidf_vectorizer = TfidfVectorizer()
    data_feature = tfidf_vectorizer.fit_transform(words)
    return data_feature, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train.tolist())
X_test_tfidf = tfidf_vectorizer.transform(X_test.tolist())

In [ ]:
X_train_tfidf.shape

# **Model 1st. LogisticRegression**

In [ ]:
lr_tfidf = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', n_jobs = -1)
lr_tfidf.fit(X_train_tfidf, y_train)
y_predicted_lr = lr_tfidf.predict(X_test_tfidf)

In [ ]:
def score_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted)
    recall = recall_score(y_test, y_predicted)
    print("accuracy = %0.3f, precision = %0.3f, recall = %0.3f" % (accuracy, precision, recall))

In [ ]:
score_metrics(y_test, y_predicted_lr)

In [ ]:
def plot_confusion_matrix(y_test, y_predicted, title='Confusion Matrix'):
    cm = confusion_matrix(y_test, y_predicted)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm,annot=True, fmt='.20g')
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
plot_confusion_matrix(y_test, y_predicted_lr)

In [ ]:
# fail to sort and plot the top 10 most important features in disaster and non-disaster text
'''
index_to_word = [(v,k) for k,v in tfidf_vectorizer.vocabulary_.items()]
sorted(index_to_word, key=lambda x: x[0], reverse=True)
'''

# **Model 2nd. DecisionTree**

In [ ]:
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier(splitter='random', class_weight='balanced'))
])
parameters = {
    'clf__max_depth':(150,160,165),
    'clf__min_samples_split':(18,20,23),
    'clf__min_samples_leaf':(5,6,7)
}

df_tfidf = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=-1, scoring='f1')
df_tfidf.fit(X_train_tfidf, y_train)

print(df_tfidf.best_estimator_.get_params())

In [ ]:
y_predicted_dt = df_tfidf.predict(X_test_tfidf)

In [ ]:
score_metrics(y_test, y_predicted_dt)

In [ ]:
plot_confusion_matrix(y_test, y_predicted_dt)

# **Feature Engineering 2nd. Word2vec**

In [ ]:
!pip install gensim -i http://pypi.douban.com/simple --trusted-host pypi.douban.com

In [ ]:
import requests
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = url.split('/')[-1]
r = requests.get(url)
with open(filename, "wb") as file:
        file.write(r.content)
        
!ls

Download the trained word2vec vectors.

In [ ]:
stop_words = stopwords.words('english')
for word in ['us','no','yet']:
    stop_words.append(word)

data_list = []
text_series = data['text']
for i in range(len(text_series)):
    content = text_series.iloc[i]
    cutwords = [word for word in content.split(' ') if word not in  stop_words if len(word) != 0]
    data_list.append(cutwords)

Get rid of stopwords.

In [ ]:
for i in range(len(data_list)):
    content = data_list[i]
    if len(content) <1:
        print(i)

In [ ]:
data_list[7626]

After deleting the stopwords, some context became null. So, when dealing with the next step vectorization, I need to change these text to zero vector. Or, at last I put feature matrix into the model will fail because of different dimension. 

In [ ]:
'''
starttime = time.time()
word2vec_model = Word2Vec(data_list, size=300, iter=10, min_count=10)
usedtime = time.time() - starttime
print('It took %.2fseconds to train word2vec' %usedtime)
'''

In [ ]:
import gensim
word2vec_path='./GoogleNews-vectors-negative300.bin.gz'
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
word2vec_model.wv['earthquake'].shape

In [ ]:
def get_textVector(data_list, word2vec, textsVectors_list):
    for i in range(len(data_list)):
        words_perText = data_list[i]
        if len(words_perText) < 1:
            words_vector = [np.zeros(300)]
        else:
            words_vector = [word2vec.wv[k]  if k in word2vec_model else  np.zeros(300) for k in words_perText]
        text_vector = np.array(words_vector).mean(axis=0)
        textsVectors_list.append(text_vector)
    return textsVectors_list

The reasons for writing  'else  np.zeros(300)' is that some words cannot be vectorized by word2vec then becoming nan.

In [ ]:
textsVectors_list = []
get_textVector(data_list, word2vec_model, textsVectors_list)
X = np.array(textsVectors_list)

Let's have a check.

In [ ]:
pd.isnull(X).any()

In [ ]:
word2vec_X = X[0:train_data.shape[0]]
y = data['target'][0:train_data.shape[0]]
word2vec_submit = X[train_data.shape[0]:-1]

X_train_word2vec, X_test_word2vec, y_train_word2vec, y_test_word2vec = train_test_split(word2vec_X, y,
                                                   test_size = 0.2, random_state = 4)

In [ ]:
print(X_train_word2vec.shape, y_train_word2vec.shape)

In [ ]:
word2vec_lr = LogisticRegression(class_weight = 'balanced', solver = 'lbfgs', n_jobs = -1)
word2vec_lr.fit(X_train_word2vec, y_train_word2vec)
y_predicted_word2vec_lr = word2vec_lr.predict(X_test_word2vec)

In [ ]:
score_metrics(y_test_word2vec, y_predicted_word2vec_lr)

In [ ]:
plot_confusion_matrix(y_test_word2vec, y_predicted_word2vec_lr)

Less 1(disaster) predicted to be 0(no-disaster). But more 0(no-disaster) predicted to be 1.

Explore the wrong predicted label.

In [ ]:
compare_list = []
for (i,j) in zip(y_test_word2vec, y_predicted_word2vec_lr):
    k = i - j
    compare_list.append(k)

wrong_num = [i for i,j in enumerate(compare_list) if j != 0]
text_series[0:train_data.shape[0]][wrong_num]

# **Model 3rd. CNN**

**Embedding + CNN**

In [ ]:
lenlen = []
for i in range(len(data_list)):
    content = data_list[i]
    perlen = len(content)
    lenlen.append(perlen)
print(max(lenlen))

In [ ]:
max_sequence_length = 26
embedding_dim = 300

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_list)
sequences = tokenizer.texts_to_sequences(data_list)
word_index = tokenizer.word_index
cnn_data = pad_sequences(sequences, maxlen = max_sequence_length)
cnn_label = to_categorical(np.asarray(train_data['target']))
print('len of word_index:', len(word_index))
print('shape of data tensor:', cnn_data.shape)
print('shape of label tensoe:', cnn_label.shape)

In [ ]:
trainCNN_data = cnn_data[0:train_data.shape[0]]
X_train_cnn, X_test_cnn, y_train_cnn, y_test_cnn = train_test_split(trainCNN_data, cnn_label,
                                                   test_size = 0.2, random_state = 4)
X_cnn, X_val_cnn, y_cnn, y_val_cnn = train_test_split(X_train_cnn, y_train_cnn,
                                                   test_size = 0.2, random_state = 4)

In [ ]:
CNNmodel = Sequential()
CNNmodel.add(Embedding(len(word_index)+1, embedding_dim, input_length = max_sequence_length))
CNNmodel.add(Conv1D(filters=250, kernel_size=3, strides=1, padding='valid', activation = 'relu'))
CNNmodel.add(MaxPooling1D(pool_size=3))
CNNmodel.add(Flatten())
CNNmodel.add(Dense(embedding_dim, activation='relu'))
CNNmodel.add(Dropout(0.8))
CNNmodel.add(Dense(cnn_label.shape[1], activation='sigmoid'))

CNNmodel.summary()

In [ ]:
CNNmodel.compile(optimizer='adam', loss=losses.binary_crossentropy, metrics=['accuracy'])
history = CNNmodel.fit(X_cnn, y_cnn, epochs=3, validation_data=(X_val_cnn, y_val_cnn))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5,1])
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = CNNmodel.evaluate(X_test_cnn, y_test_cnn, verbose=2)
print('test loss:',test_loss)
print('test acc:',test_acc)

**Word2Vec + CNN**

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items(): 
    if word in word2vec_model:
        embedding_matrix[i] = np.asarray(word2vec_model.wv[word])

In [ ]:
embedding_layer = Embedding(len(word_index)+1,
                           embedding_dim,
                           weights = [embedding_matrix],
                           input_length = max_sequence_length,
                           trainable = False)

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(filters=150, kernel_size=3, strides=1, padding='valid', activation = 'relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(embedding_dim, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(cnn_label.shape[1], activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss=losses.binary_crossentropy, metrics=['accuracy'])
history = model.fit(X_cnn, y_cnn, epochs=10, validation_data=(X_val_cnn, y_val_cnn))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5,1])
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test_cnn, y_test_cnn, verbose=2)
print('test loss:',test_loss)
print('test acc:',test_acc)

# **Model 4th. TFhub+Bert**

*Ensure tensorflow's version >= 2.0 & tensorflow_hub's version >= 0.5.0*

In [ ]:
tf.__version__

In [ ]:
import tensorflow_hub as hub
hub.__version__

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
# get the official tokenization created by the Google team (??)

* *I have a question: If I do not get the official tokenization script here, 'tokenizer.tokenize' in get_tokens function will returns error:'raise _exceptions.UnparsedFlagAccessError(error_message)--Trying to access flag ... before flags were parsed'.*
* *If anyone konws the reasons, please tell me. THANKS!*

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

import tokenization

In [ ]:
def bert_encode(texts, bert_layer, max_len=128):
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)
    
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len - 2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        input_ids = tokens + [0]* pad_len
        all_tokens.append(input_ids)

        masks = [1]*len(input_sequence) + [0]* pad_len
        all_masks.append(masks)
        
        segments = [0]* max_len
        all_segments.append(segments)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

    
def build_model(bert_layer, max_len = 128, lr = 1e-5):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32,name="input_word_ids")
    input_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32,name="input_mask")
    segment_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32,name="segment_ids")
        
    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    dense_out = Dense(1,activation="relu")(pooled_output)
    drop_out = tf.keras.layers.Dropout(0.8)(dense_out)
    out = Dense(1,activation="sigmoid")(pooled_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    adam = tf.keras.optimizers.Adam(lr)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
        
    return model


def plot_curve(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5,1])
    plt.legend()
    plt.show()

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
# read and encode train data
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

train_input = bert_encode(train.text.values, bert_layer, max_len=128)
train_labels = np.array(train.target)

*Here I used the data without cleaning.*

In [ ]:
# train model
model = build_model(bert_layer, max_len=128, lr = 1e-5)
model.summary()

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

train_history = model.fit(
    train_input, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=16
)

plot_curve(train_history)

*Obviously, the model is overfitting here. If anyone knows nice tricks to adjust overfitting, please tell me. Thanks!*

In [ ]:
# predict
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

test_input = bert_encode(test.text.values, bert_layer, max_len=128)
model.load_weights('model.h5')
test_pred = model.predict(test_input)

In [ ]:
# submit
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission['target'] = np.round(test_pred).astype('int')
submission.to_csv('submission.csv', index=False)
submission.groupby('target').count()